In [1]:
%matplotlib inline
import os, io, json, urllib, numpy as np, codecs
import glob, pandas as pd
pd.options.display.max_rows = 999

In [24]:
def h_(citation):
    citation=list(citation)
    h=0
    citation.sort(reverse=True)
    for i in range(1,len(citation)+1):
        if citation[i-1] >= i:
            h=i
    return h

In [369]:
# skip if result.txt exists
y16 = glob.glob('data/2016/*.txt')
y15 = glob.glob('data/2015/*.txt')
y14 = glob.glob('data/2014/*.txt')
y14_16=y14+y15+y16
with open("data/data.txt", "wb") as outfile:
    for f in y14_16:
        with open(f, "rb") as infile:
            lines=infile.read().split('\n')
            for line in lines:
                if len(line.split('\t'))==10:
                    outfile.write(line+'\n')

In [389]:
df=pd.read_table('data/data.txt',header=None)
df.columns=['doi','year','ref','citation','issnp','issne','journal','publisher','author','lic']
df.shape

(9122830, 10)

In [371]:
#df=df[df['author']==1]
#df.shape

In [372]:
df['year']=df['year'].astype(int)

In [373]:
df=df[df['year']<=2016][df['year']>=2014]
df.shape

C:\Users\moqri\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


(9057116, 10)

In [374]:
df['count']=1
df['cited']=df['citation'].apply(lambda x: 1 if x>0 else 0)
df['un-cited']=df['citation'].apply(lambda x: 0 if x>0 else 1)

In [375]:
df=df[(~df.issnp.isnull()) | (~df.issne.isnull())]
df.shape

(9051765, 13)

In [376]:
df=df[['year','count','citation','issnp','issne']].fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')

In [393]:
df['issn']=df.issn.apply(lambda x:x[0:9] if x[10:19]==x[0:9] else x)

In [394]:
s=df.issn.str.split(',',expand=True).stack()

In [395]:
i = s.index.get_level_values(0)

In [396]:
df2 = df.loc[i].copy()

In [397]:
df2["@id"] = s.values

In [382]:
dg=df2.groupby(['@id','year']).agg({'count':sum,'citation':sum}).reset_index()
dg['impact']=dg['citation'].div(dg['count']).round(2)
dg.shape

(151694, 5)

In [383]:
dg_h=df2[['@id','citation']].groupby(['@id']).agg(lambda x:tuple(x)).reset_index()
dg_h['h']=dg_h.citation.apply(lambda x: h_(x))
dg_h.shape

(59084, 3)

In [398]:
dg_i=df2.groupby(['@id']).agg({'count':sum,'citation':sum}).reset_index()
dg_i['impact']=dg_i['citation'].div(dg_i['count']).round(2)
dg_i.shape

KeyError: 'count'

In [385]:
dg_flat=dg.pivot(index='@id',columns='year')
dg_flat.columns = [col[0]+'_'+str(col[1])[3] for col in dg_flat.columns.values]
dg_flat=dg_flat.reset_index()

In [386]:
dg_merge=dg_flat.merge(dg_i[['@id','count','citation','impact']])\
.merge(dg_h[['@id','h']]).fillna(0,downcast='infer')
dg_merge.shape

(59084, 14)

In [388]:
for s in range(10):
    dic=[]
    for index, i in dg_merge[10000*s:10000*(s+1)].iterrows():
        journal={}
        journal['@id']=i['@id']
        journal['e:count_4']=i['count_4']
        journal['e:citation_4']=i['citation_4']
        journal['e:impact_4']=round(i['impact_4'],2)
        journal['e:count_5']=i['count_5']
        journal['e:citation_5']=i['citation_5']
        journal['e:impact_5']=round(i['impact_5'],2)
        journal['e:count_6']=i['count_6']
        journal['e:citation_6']=i['citation_6']
        journal['e:impact_6']=round(i['impact_6'],2)
        journal['e:count']=i['count']
        journal['e:citation']=i['citation']
        journal['e:impact']=round(i['impact'],2)
        journal['e:h']=i['h']
        if journal not in dic:
            dic.append(journal)
    with open('jsonld/'+str(s)+'journals_stats.jsonld', 'w') as outfile:
        json.dump(dic, outfile)

In [392]:
df=df.fillna('_')
df['issn']=df.issnp+','+df.issne
df['issn']=df.issn.str.replace(',_','')
df['issn']=df.issn.str.replace('_,','')

In [401]:
df2

,doi,year,ref,citation,issnp,issne,journal,publisher,author,lic,issn,@id
0,10.15290/bsp.2014.17.04,2014,0,0,1689-7404,_,Białostockie Studia Prawnicze,University of Bialystok,1,www.elsevier.com,1689-7404,1689-7404
1,10.1016/j.jpedsurg.2014.09.045,2014,0,0,0022-3468,_,Journal of Pediatric Surgery,Elsevier BV,1,www.elsevier.com,0022-3468,0022-3468
2,10.1016/j.jpedsurg.2014.09.050,2014,0,0,0022-3468,_,Journal of Pediatric Surgery,Elsevier BV,1,www.elsevier.com,0022-3468,0022-3468
3,10.1136/vr.g7231,2014,0,0,0042-4900,2042-7670,Veterinary Record,BMJ,0,_,"0042-4900,2042-7670",0042-4900
3,10.1136/vr.g7231,2014,0,0,0042-4900,2042-7670,Veterinary Record,BMJ,0,_,"0042-4900,2042-7670",2042-7670
4,10.1186/preaccept-1025186331138691,2014,0,0,1472-6874,_,BMC Women's Health,Springer Nature,1,_,1472-6874,1472-6874
5,10.1186/preaccept-1248701577135074,2014,0,0,1756-9966,_,Journal of Experimental & Clinical Cancer Rese...,Springer Nature,1,_,1756-9966,1756-9966
6,10.1186/preaccept-1255381821380460,2014,0,0,1471-2350,_,BMC Medical Genetics,Springer Nature,1,_,1471-2350,1471-2350
7,10.1186/preaccept-1310416131408627,2014,0,0,1756-3305,_,Parasites & Vectors,Springer Nature,1,_,1756-3305,1756-3305
8,10.1186/preaccept-1367363142137775,2014,0,0,2045-7022,_,Clinical and Translational Allergy,Springer Nature,1,_,2045-7022,2045-7022
